In [1]:
import pandas as pd
import os
from datetime import date, datetime, timedelta
import numpy as np
import seaborn as sns

C:\Users\marie\AppData\Local\Temp\ipykernel_2348\1657975687.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def interval_seconds(interval_str):
    if len(interval_str.split(" d ")) > 1:
        days = int(interval_str.split(" d ")[0])
        times = interval_str.split(" d ")[1].split(":")
    else:
        days = 0
        times = interval_str.split(" d ")[-1].split(":")
    hours = int(times[0])
    minutes = int(times[1])
    seconds = int(times[2])
    interval = timedelta(days = days, hours = hours, minutes = minutes, seconds = seconds).total_seconds()
    return interval

In [7]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "data")

df = pd.read_csv(os.path.join(data_path, "delpro_visit_lactation.csv")) # , header = 0,  names = names, dtype = dtypes)
df = df[df['date'].notna()]
# remove visits with yield 0 
df = df[df["incomplete_milkings"] == 0].reset_index(drop = True)
df = df[df["milk_flow_duration"] > 0 ].reset_index(drop = True)
# date formats: date, begin time, end time
df.date = [datetime.strptime(day, "%Y-%m-%d") for day in df.date]
df.begin_time = [datetime.strptime(day, "%m/%d/%Y %I:%M %p") for day in df.begin_time]
df.end_time = [datetime.strptime(day, "%I:%M %p") for day in df.end_time]
df["end_time2"] = [rec[2] + timedelta(days=1, hours = rec[1].time().hour, minutes = rec[1].time().minute) \
                   if rec[0].time() > rec[1].time() else rec[2] + timedelta(days=0, hours = rec[1].time().hour, minutes = rec[1].time().minute) \
                   for rec in zip(df.begin_time, df.end_time, df.date)]
df["total_visit_duration"] = [stamp.total_seconds() for stamp in  (df.end_time2 - df.begin_time)]
df["notmilk_visit_duration"] = [num for num in (df.total_visit_duration - df.milk_flow_duration)]
df["interval_sec"] = [interval_seconds(rec) for rec in df.interval]

agg_df = df[["animal_number", "notmilk_visit_duration", "milk_flow_duration", "date", \
             "yield", "yield_lf", "yield_lr", "yield_rr", "yield_rf",\
            "peak_flow_lf", "peak_flow_lr", "peak_flow_rr", "peak_flow_rf", "interval_sec"]]\
    .groupby("animal_number")\
    .agg(milk_flow_duration_mean = ('milk_flow_duration', 'mean'),\
         yield_visit_mean = ('yield', 'mean'),\
         yield_lf_visit_mean = ('yield_lf', 'mean'),\
         yield_lr_visit_mean = ('yield_lr', 'mean'),\
         yield_rr_visit_mean = ('yield_rr', 'mean'),\
         yield_rf_visit_mean = ('yield_rf', 'mean'),\
         peak_flow_rf_mean = ('peak_flow_rf', 'mean'),\
         peak_flow_lf_mean = ('peak_flow_lf', 'mean'),\
         peak_flow_rr_mean = ('peak_flow_rr', 'mean'),\
         peak_flow_lr_mean = ('peak_flow_lr', 'mean'),\
         notmilk_visit_duration_mean = ("notmilk_visit_duration",'mean'),\
         notmilk_visit_duration_std = ("notmilk_visit_duration",'std'),\
         interval_sec_mean = ("interval_sec", 'mean')).reset_index()

agg_df.to_csv(os.path.join(data_path, "duration_lactation.csv"), index = False)

In [ ]:
sns.scatterplot(data=agg_df, x="notmilk_visit_duration", y="milk_flow_duration")

In [ ]:
sns.scatterplot(data=df, x="days_in_milk", y="milk_flow_duration")

In [ ]:
sns.scatterplot(data=df, x="yield", y="milk_flow_duration")

In [ ]:
df[df["incomplete_milkings"] ==1]